In [13]:
import keras
import numpy as np
from keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential

In [4]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

600901/600901 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
Corpus length: 600893


In [7]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [14]:
model = Sequential([
    Input(shape=(maxlen, len(chars))),
    LSTM(128),
    Dense(len(chars), activation='softmax')
])

In [16]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [17]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [19]:
import random
import sys

for epoch in range(1, 10):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 99s 64ms/step - loss: 1.9614
--- Generating with seed: "onal inasmuch as each of the combatants may say: "either i
c"
------ temperature: 0.2
onal inasmuch as each of the combatants may say: "either i
condection of the sand many and the sand and and the self for the self and the self and the same for the sand and still of the saction of the sand the self and the moral and the sand the say and the preass of the part of the world of the fact man in the plan condected and the interptation of the self and self of the most of the fact as the sand and the self and the was and the world and the self an
------ temperature: 0.5
 sand and the self and the was and the world and the self and soul and sour a stare
symplents the preasinal conscience of and
formaning in the pretaye in which the same to be and speciation and something the generally the interpained of him sporice
believe the sang and perpossion of say, and how the all to many the means the in the 

KeyboardInterrupt: 